# Experiment 1a - Process tactile image data
* processed tactile data saved into two arrays:
*        feature_SA and features_RA of shape (n_samples, n_pins, n_pins)
* n_pins is number of pins along side of square array (here 19)
* n_samples is here over time during a tapping contact
* values of features are the SA and FA firing rates from module afferent_response.py
* overall, the processing is iterated as a list over the data sets ('slow' and 'fast' contacts in this case)

To run, first edit dir_data to path where data stored.

In [14]:
import os, sys, pickle, json
import numpy as np

sys.path.append(os.path.abspath('../common'))
from afferent_response import AfferentResponse as ar

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def open_obj(name):
    with open(name + '.pkl', 'rb') as f:
        obj = pickle.load(f)
    return obj

In [15]:
dir_data = os.environ["DATAPATH"] + "/open/afferents-tactile-gratings-jrsi2022/experiment_1a"

sets = ["slow", "fast"]
n_sets = len(sets)

with open("../common/tip_params/tipA", 'r') as read_file:
    tip_params = json.load(read_file)

In [16]:
def rotate_pins(pins, theta):
    R = np.array(((np.cos(theta), -np.sin(theta)), (np.sin(theta), np.cos(theta))))
    pins_rot = [np.matmul(R, pin) for pin in pins]
    return np.array(pins_rot)

def locate_pins(pins, n_pins):
    locs = np.zeros((n_pins, n_pins))
    index_xSort = np.argsort(pins[:,0], axis=0)
    
    index_ySort = np.zeros((n_pins, n_pins))
    n = 0
    for k in range(n_pins):
        indices = np.zeros((n_pins,2))
        for l in range(n_pins):
            indices[l,:] = pins[index_xSort[n],:]
            n+=1
        index_ySort[k,:] = np.argsort(indices[:,1], axis=0)

    for k in range(n_pins):
        for l in range(n_pins):
            locs[k,l] = index_xSort[int(index_ySort[k,l])+n_pins*k]
    return locs

def array_pins(pins, locs):
    n1_pins, n2_pins = np.shape(locs)
    pins_array = np.zeros((2, n1_pins, n2_pins))
    for k in range(n1_pins):
        for l in range(n2_pins):
            pins_array[:,k,l] = pins[int(locs[k,l]),:]  
    return pins_array

def features(firing, locs):
    n1_pins, n2_pins = np.shape(locs)
    n_samples = np.shape(firing)[0]
    features = np.zeros((n_samples, n1_pins, n2_pins))
    for k in range(n1_pins):
        for l in range(n2_pins):
            features[:,k,l] = firing[:, int(locs[k,l])]     
    return features

In [17]:
afferents = ar(False, **tip_params)
pins_init = afferents.init_pins(dir_data + rf"/{sets[0]}/throw_away.mp4")
print(f"n_pins={len(pins_init)}")

firing_SA = []; firing_RA = []; pins = []
for i in range(n_sets):
    firing = afferents.firing(dir_data + rf"/{sets[i]}/x_0.mp4", pins_init)

    firing_SA.append(np.stack([firing[0][f'{k}'] for k in range(len(firing[0]))]))
    firing_RA.append(np.stack([firing[1][f'{k}'] for k in range(len(firing[1]))]))
    pins.append(firing[2])

In [18]:
theta = np.radians(-6.2)
n_pins = 19

pins_rot = [rotate_pins(pins[i], theta) for i in range(n_sets)]
locs_rot = [locate_pins(pins_rot[i], n_pins) for i in range(n_sets)]
pins_array = [array_pins(pins_rot[i], locs_rot[i]) for i in range(n_sets)]

save_obj(pins_array, dir_data + r"/pins")

features_SA = [features(firing_SA[i], locs_rot[i]) for i in range(n_sets)]
features_RA = [features(firing_RA[i], locs_rot[i]) for i in range(n_sets)]

save_obj(features_SA, dir_data + r"/features_SA")
save_obj(features_RA, dir_data + r"/features_RA")